In [133]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer

import os
import json
import datetime
import openpyxl
import time
import random

import numpy as np
import pandas as pd

from sklearn.utils import shuffle

In [134]:
df = pd.read_excel("Trainingsdaten_mit_Abnahmeprotokoll_neu_2.xlsx")
df.columns = list(df)
df[50:60]

,Benennung,Baumaßnahme,Gewährleistungsdauer
50,Brandmeldeanlage,Aufrüstung der Brandmeldeanlage,24
51,Kältetechnische Anlage,Austausch Dreiwegeventil am PWT-Eintritt,24
52,sanitäre Anlagen,Austausch der Trinkwasserleitung in der Garage...,48
53,Hebeanlage,Erneuerung der Druckschläuche ; Denka-Lift,48
54,Lufttechnische Anlagen,Austausch Klima Innengeräte F0 / 134,24
55,Kältetechnische Anlage,Austausch Dreiwegeventil am PWT - Eintritt Eis...,24
56,Gebäudeautomation,Austausch der Heizungsregelung im Geb. 5 / 6,24
57,Gebäudeautomation,Austausch der Heizungsregelung im Geb. 5 / 6,24
58,Lufttechnische Anlagen,Austausch eines defekten Abluftmotors LA 2.27 CCC,24
59,Gebäudeautomation,Austausch der Heizungsregelung im Geb. 5 / 6,24


In [135]:
text = df[["Benennung","Baumaßnahme"]].values
targets = df["Gewährleistungsdauer"].values

data = []

text, targets = shuffle(text, targets, random_state=0)
text, targets = shuffle(text, targets, random_state=0)

for i in range(len(text)):
    text_input = [text[i][0] + " " + text[i][1]]
    data.append({"class":str(targets[i]), "sentence":str(text_input)})
    

train_len = 350
    
training_data = data[0:train_len]
test_data = data[train_len:]

In [136]:
words = []
classes = []
documents = []
ignore_words = ['?',':','!',',',';','[',']']
stemmer = LancasterStemmer()

# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern['sentence'])
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words, words 0-50: ", words[0:50])

350 documents
2 classes ['24', '48']
862 unique stemmed words, words 0-50:  ["'heizungsanlage", 'e', 'fit', 'kg', 'von', 'aufzüg', 'schwachstrom', 'beleuchtung', "'kita", 'fahrzeughal', 'treppenhäusern', "'überwachung", 'rollt', '95/1741', "'lüftung", 'unterstützungspfost', 'flüssigboden', 'umkleid', 'sekundärpump', 'city', "'rackanschluss", "'ertüchtigung", 'drahtseil', "'kanalsanierung", 'schiebetorantrieb', 'nachrüstung', '95/1732', '134', 'brandabschnit', 'bsk', "'putz-", 'bruchs', "'blitzschutz", 'absperrschieb', 'hydrauliksteuerungsblock', "'dachinstandsetzung", 'verkabelung', 'hkls', "'umkehrosmose", 'fernwärm', 'sanierung', 'vsg-glasscheibe', 'splitgeräten', 'fluchttür', 'nea', 'notraffsystem', 'ibec', 'radlerdusch', 'das', 'og']


In [157]:
# create our training data
X_data = []
Y_data = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    X_data.append(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    Y_data.append(output_row)
    
X_train = np.array(X_data).astype(float)
Y_train = np.array(Y_data).astype(float)

In [189]:
import tensorflow as tf
from tensorflow.keras import layers

n_features = X_train[0].shape

GWM_classifier = tf.keras.Sequential()
GWM_classifier.add(layers.Dense(64, activation='relu', input_dim=X_train[0].shape[0]))
GWM_classifier.add(layers.Dense(2, activation='softmax'))

GWM_classifier.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

In [190]:
GWM_classifier.fit(X_train, Y_train, validation_split=0.20, shuffle=True, epochs=500)

Train on 280 samples, validate on 70 samples
Epoch 1/500
280/280 [==============================] - 0s 1ms/sample - loss: 0.6913 - acc: 0.5393 - val_loss: 0.6974 - val_acc: 0.4714
Epoch 2/500
280/280 [==============================] - 0s 96us/sample - loss: 0.6812 - acc: 0.6107 - val_loss: 0.6933 - val_acc: 0.4857
Epoch 3/500
280/280 [==============================] - 0s 114us/sample - loss: 0.6730 - acc: 0.6214 - val_loss: 0.6899 - val_acc: 0.5286
Epoch 4/500
280/280 [==============================] - 0s 107us/sample - loss: 0.6657 - acc: 0.6250 - val_loss: 0.6872 - val_acc: 0.5286
Epoch 5/500
280/280 [==============================] - 0s 121us/sample - loss: 0.6592 - acc: 0.6321 - val_loss: 0.6850 - val_acc: 0.5571
Epoch 6/500
280/280 [==============================] - 0s 107us/sample - loss: 0.6537 - acc: 0.6286 - val_loss: 0.6831 - val_acc: 0.5571
Epoch 7/500
280/280 [==============================] - 0s 103us/sample - loss: 0.6487 - acc: 0.6214 - val_loss: 0.6814 - val_acc: 0.5571

280/280 [==============================] - 0s 107us/sample - loss: 0.4925 - acc: 0.7893 - val_loss: 0.6062 - val_acc: 0.6143
Epoch 61/500
280/280 [==============================] - 0s 114us/sample - loss: 0.4896 - acc: 0.8000 - val_loss: 0.6046 - val_acc: 0.6143
Epoch 62/500
280/280 [==============================] - 0s 103us/sample - loss: 0.4868 - acc: 0.8036 - val_loss: 0.6030 - val_acc: 0.6143
Epoch 63/500
280/280 [==============================] - 0s 107us/sample - loss: 0.4841 - acc: 0.8143 - val_loss: 0.6016 - val_acc: 0.6143
Epoch 64/500
280/280 [==============================] - 0s 103us/sample - loss: 0.4814 - acc: 0.8179 - val_loss: 0.5999 - val_acc: 0.6143
Epoch 65/500
280/280 [==============================] - 0s 100us/sample - loss: 0.4783 - acc: 0.8250 - val_loss: 0.5983 - val_acc: 0.6143
Epoch 66/500
280/280 [==============================] - 0s 114us/sample - loss: 0.4757 - acc: 0.8286 - val_loss: 0.5969 - val_acc: 0.6286
Epoch 67/500
280/280 [=========================

280/280 [==============================] - 0s 107us/sample - loss: 0.3459 - acc: 0.8821 - val_loss: 0.5465 - val_acc: 0.6857
Epoch 120/500
280/280 [==============================] - 0s 107us/sample - loss: 0.3442 - acc: 0.8821 - val_loss: 0.5460 - val_acc: 0.6857
Epoch 121/500
280/280 [==============================] - 0s 110us/sample - loss: 0.3418 - acc: 0.8821 - val_loss: 0.5455 - val_acc: 0.6857
Epoch 122/500
280/280 [==============================] - 0s 100us/sample - loss: 0.3400 - acc: 0.8821 - val_loss: 0.5449 - val_acc: 0.6857
Epoch 123/500
280/280 [==============================] - 0s 110us/sample - loss: 0.3378 - acc: 0.8893 - val_loss: 0.5447 - val_acc: 0.6857
Epoch 124/500
280/280 [==============================] - 0s 110us/sample - loss: 0.3358 - acc: 0.8857 - val_loss: 0.5443 - val_acc: 0.6857
Epoch 125/500
280/280 [==============================] - 0s 107us/sample - loss: 0.3337 - acc: 0.8857 - val_loss: 0.5441 - val_acc: 0.6857
Epoch 126/500
280/280 [==================

280/280 [==============================] - 0s 100us/sample - loss: 0.2456 - acc: 0.9357 - val_loss: 0.5398 - val_acc: 0.7286
Epoch 179/500
280/280 [==============================] - 0s 107us/sample - loss: 0.2443 - acc: 0.9393 - val_loss: 0.5400 - val_acc: 0.7286
Epoch 180/500
280/280 [==============================] - 0s 103us/sample - loss: 0.2427 - acc: 0.9357 - val_loss: 0.5401 - val_acc: 0.7286
Epoch 181/500
280/280 [==============================] - 0s 107us/sample - loss: 0.2416 - acc: 0.9429 - val_loss: 0.5402 - val_acc: 0.7286
Epoch 182/500
280/280 [==============================] - 0s 103us/sample - loss: 0.2402 - acc: 0.9357 - val_loss: 0.5405 - val_acc: 0.7286
Epoch 183/500
280/280 [==============================] - 0s 100us/sample - loss: 0.2389 - acc: 0.9393 - val_loss: 0.5407 - val_acc: 0.7286
Epoch 184/500
280/280 [==============================] - 0s 103us/sample - loss: 0.2374 - acc: 0.9429 - val_loss: 0.5408 - val_acc: 0.7286
Epoch 185/500
280/280 [==================

280/280 [==============================] - 0s 103us/sample - loss: 0.1779 - acc: 0.9750 - val_loss: 0.5567 - val_acc: 0.7143
Epoch 238/500
280/280 [==============================] - 0s 103us/sample - loss: 0.1770 - acc: 0.9750 - val_loss: 0.5570 - val_acc: 0.7143
Epoch 239/500
280/280 [==============================] - 0s 100us/sample - loss: 0.1761 - acc: 0.9786 - val_loss: 0.5574 - val_acc: 0.7143
Epoch 240/500
280/280 [==============================] - 0s 103us/sample - loss: 0.1750 - acc: 0.9786 - val_loss: 0.5579 - val_acc: 0.7143
Epoch 241/500
280/280 [==============================] - 0s 103us/sample - loss: 0.1741 - acc: 0.9786 - val_loss: 0.5583 - val_acc: 0.7143
Epoch 242/500
280/280 [==============================] - 0s 107us/sample - loss: 0.1733 - acc: 0.9786 - val_loss: 0.5588 - val_acc: 0.7143
Epoch 243/500
280/280 [==============================] - 0s 107us/sample - loss: 0.1722 - acc: 0.9786 - val_loss: 0.5591 - val_acc: 0.7143
Epoch 244/500
280/280 [==================

280/280 [==============================] - 0s 107us/sample - loss: 0.1314 - acc: 0.9893 - val_loss: 0.5835 - val_acc: 0.7143
Epoch 297/500
280/280 [==============================] - 0s 103us/sample - loss: 0.1307 - acc: 0.9893 - val_loss: 0.5840 - val_acc: 0.7143
Epoch 298/500
280/280 [==============================] - 0s 107us/sample - loss: 0.1300 - acc: 0.9893 - val_loss: 0.5845 - val_acc: 0.7143
Epoch 299/500
280/280 [==============================] - 0s 107us/sample - loss: 0.1294 - acc: 0.9893 - val_loss: 0.5850 - val_acc: 0.7143
Epoch 300/500
280/280 [==============================] - 0s 100us/sample - loss: 0.1287 - acc: 0.9893 - val_loss: 0.5856 - val_acc: 0.7143
Epoch 301/500
280/280 [==============================] - 0s 100us/sample - loss: 0.1282 - acc: 0.9893 - val_loss: 0.5861 - val_acc: 0.7143
Epoch 302/500
280/280 [==============================] - 0s 100us/sample - loss: 0.1275 - acc: 0.9929 - val_loss: 0.5866 - val_acc: 0.7143
Epoch 303/500
280/280 [==================

280/280 [==============================] - 0s 118us/sample - loss: 0.0993 - acc: 0.9929 - val_loss: 0.6149 - val_acc: 0.7143
Epoch 356/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0988 - acc: 0.9929 - val_loss: 0.6154 - val_acc: 0.7000
Epoch 357/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0984 - acc: 0.9929 - val_loss: 0.6160 - val_acc: 0.7000
Epoch 358/500
280/280 [==============================] - 0s 96us/sample - loss: 0.0979 - acc: 0.9929 - val_loss: 0.6165 - val_acc: 0.7000
Epoch 359/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0975 - acc: 0.9929 - val_loss: 0.6170 - val_acc: 0.7000
Epoch 360/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0971 - acc: 0.9929 - val_loss: 0.6175 - val_acc: 0.7000
Epoch 361/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0966 - acc: 0.9929 - val_loss: 0.6180 - val_acc: 0.7000
Epoch 362/500
280/280 [===================

280/280 [==============================] - 0s 107us/sample - loss: 0.0770 - acc: 1.0000 - val_loss: 0.6460 - val_acc: 0.6857
Epoch 415/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0766 - acc: 1.0000 - val_loss: 0.6466 - val_acc: 0.6857
Epoch 416/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0763 - acc: 1.0000 - val_loss: 0.6472 - val_acc: 0.6714
Epoch 417/500
280/280 [==============================] - 0s 100us/sample - loss: 0.0760 - acc: 1.0000 - val_loss: 0.6476 - val_acc: 0.6714
Epoch 418/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0757 - acc: 1.0000 - val_loss: 0.6482 - val_acc: 0.6714
Epoch 419/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0754 - acc: 1.0000 - val_loss: 0.6486 - val_acc: 0.6714
Epoch 420/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0751 - acc: 1.0000 - val_loss: 0.6492 - val_acc: 0.6857
Epoch 421/500
280/280 [==================

280/280 [==============================] - 0s 100us/sample - loss: 0.0612 - acc: 1.0000 - val_loss: 0.6765 - val_acc: 0.6714
Epoch 474/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0610 - acc: 1.0000 - val_loss: 0.6770 - val_acc: 0.6714
Epoch 475/500
280/280 [==============================] - 0s 103us/sample - loss: 0.0607 - acc: 1.0000 - val_loss: 0.6775 - val_acc: 0.6714
Epoch 476/500
280/280 [==============================] - 0s 110us/sample - loss: 0.0605 - acc: 1.0000 - val_loss: 0.6780 - val_acc: 0.6714
Epoch 477/500
280/280 [==============================] - 0s 110us/sample - loss: 0.0603 - acc: 1.0000 - val_loss: 0.6786 - val_acc: 0.6714
Epoch 478/500
280/280 [==============================] - 0s 107us/sample - loss: 0.0600 - acc: 1.0000 - val_loss: 0.6790 - val_acc: 0.6714
Epoch 479/500
280/280 [==============================] - 0s 110us/sample - loss: 0.0598 - acc: 1.0000 - val_loss: 0.6795 - val_acc: 0.6714
Epoch 480/500
280/280 [==================

In [195]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details).reshape(1,-1)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    result = GWM_classifier.predict(x)
    return result

def classify(sentence, target, show_details=False):
    result = think(sentence, show_details=False)
    if np.argmax(result)==0:
        warranty = 24
    else:
        warranty = 48
    if show_details:
        print ("classification: %s" % warranty)
        print ("correct answer: " + target)
    return warranty

def query(sentence, show_details=False):
    results = think(sentence, show_details=False)
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

In [196]:
X_test = []
Y_test = []

for i in range(len(test_data)):
    X_test.append(test_data[i].get('sentence'))
    Y_test.append(test_data[i].get('class'))

In [201]:
ERROR_THRESHOLD = 0.2
scorecard = []

for i in range(len(test_data)):
    print("Testdatenpunkt: " + str(i))
    result = classify(X_test[i],Y_test[i], show_details=True)
    print("\n")
    if int(result) == int(Y_test[i]):
        scorecard.append("1")
    else: 
        scorecard.append("0")



Testdatenpunkt: 0
classification: 48
correct answer: 48


Testdatenpunkt: 1
classification: 48
correct answer: 48


Testdatenpunkt: 2
classification: 24
correct answer: 24


Testdatenpunkt: 3
classification: 48
correct answer: 48


Testdatenpunkt: 4
classification: 48
correct answer: 48


Testdatenpunkt: 5
classification: 48
correct answer: 48


Testdatenpunkt: 6
classification: 48
correct answer: 48


Testdatenpunkt: 7
classification: 48
correct answer: 48


Testdatenpunkt: 8
classification: 48
correct answer: 48


Testdatenpunkt: 9
classification: 48
correct answer: 48


Testdatenpunkt: 10
classification: 48
correct answer: 48


Testdatenpunkt: 11
classification: 48
correct answer: 48


Testdatenpunkt: 12
classification: 24
correct answer: 24


Testdatenpunkt: 13
classification: 48
correct answer: 48


Testdatenpunkt: 14
classification: 24
correct answer: 48


Testdatenpunkt: 15
classification: 48
correct answer: 48


Testdatenpunkt: 16
classification: 48
correct answer: 48


Testdat

In [198]:
sums = 0
for i in range(len(scorecard)):
    sums += int(scorecard[i])

accuracy = sums / len(scorecard)
print("accuracy: " +  str(accuracy))
print("length of test data: " + str(len(test_data)))

accuracy: 0.7924528301886793
length of test data: 106
